## Class
A class is most commonly a collection of function known as *methods*, variables as *class variables*, and computed attributes as *properties*.

For example, a **class** is defined as follows:

In [1]:
class Dog(object):
    num_dogs = 0
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
        Dog.num_dogs += 1
    
    def __del__(self):
        Dog.num_dogs -= 1
    
    def bark(self):
        print "Wang Wang Wang"
        
    def inquiry(self):
        return self.age

The instance method like *bark* operates on an instance of the class. Class variables such as *num_dogs* are values that are shared among all instances of a class.

## Class Instances
The class definition can be considered as the blueprint, however there are no any instances of the class. 

In [2]:
a = Dog("Tina", 2)

The above code creates a new instance of **Dog** by calling a class object as a function. The new instance *a* will be passed to the \__init\__() method of the class. 

When you access an attribute, the instance is checked first and if nothing is known, the search moves to the instance's **class** instead.

Although **classes** define a namespace, classes do not create a scope for names used inside teh bodies of methods. Therefore, the explicit use of **self** is required. 

## Inheritance
When a class is created via inheritance, it "inherits" the attributes defined by its base class. There is a root of  class called **object**, which provides the default implementation of some common methods. 

In [3]:
class OldDog(Dog):
    def inquiry(self):
        super(OldDog, self).bark()
        if self.age > 3:
            return True
        else:
            return False

In [4]:
b = OldDog("Grey", 5)
b.inquiry()

Wang Wang Wang


True

Like the example above, **super(class, instance)** returns a special object that lets you perform attribute lookups on the base class. In *Python 3*, **super()** will be used instead.

To find attributes with multiple inheritnce, all base classes are ordered in a list from the "most specialized" class to the "least specialized" class. Then searching for an attribute, this list is searched in order until the first definition of the attribute is found. 

In [5]:
OldDog.__mro__

(__main__.OldDog, __main__.Dog, object)

The ordering of base classes can be viewed by the attribute. The precise ordering of base classes is actually quite complex and not based on any simple algorithm. Multiple inheritance is best avoided in mot programs. 

Whenever an attribute is accessed as *obj.attr*, *attr* is located by searching within instance itself, then the instance's class definition, and then base class. This binding process independent of what kind of *obj* is, is considered as *duck typing* as "if it looks like, quacks like, and walks like a duck, then it's a duck"

## Descriptors
An object that represents the value of an attribute is called *a descriptor object* in Python. A **descriptor** object needs to implement one or more of special methods \__get\__(), \__set\__() and \__delete\__(), it can hook into the attribute access mechanism and can customize those operations. 

In [6]:
class TypeAttribute(object):
    def __init__(self, name, type, default = None):
        self.name = "_" + name
        self.type = type
        self.default = default if default else type()
    
    def __get__(self, instance, cls):
        return getattr(instance, self.name, self.default)
    
    def __set__(self, instance, value):
        if not isinstance(value, self.type):
            raise TypeError("Must be a %s" %self.type)
        setattr(instance, self.name, value)
    
    def __delete__(self, instance):
        raise AttributeError("Can't delete attribute")
        
class Test(object):
    name = TypeAttribute("name", str)
    age = TypeAttribute("age", int, 15)

In [7]:
t = Test()
a = t.name
t.name = "Nesta"
del t.name

AttributeError: Can't delete attribute

From the example above, one can observe that the descriptor can be set and get and the error is raised when deleted. It is **not legal** to create descriptors on a per-instance basis by creating descriptor objects inside \__init\__() or other methods in the class. And the attribute name used by the class to hold a descriptor takes precedence over attribute stored on instance. Thus different names should be supplied to avoid the conflict. 

## private attribute
To make the attributes private, names in a class should start with a **double underscores** which will be automatically mangled to form a new name of the form \__Classname\__name.

Although the name mangling in the class is the way to implement data encapsulation, there are several points should be caution when used:
* Name mangling does not occur in function such as *getattr()*, *hasattr()*, *setattr()* or *delattr()* where the attribute name is specified as a string
* Giving a method a private name is a technique that a superclass can use to prevent a derived class from redifining and changing the implementation of a method. 
* In *module*, the name with a *single* leading underscore prevents names from being exported by the **from module import \* ** statment. In *class*, this name convention *will not* hide the attribute. 

## Object Memory Management
The creation of an instance is carried out in two steps using the special method \__new\__(), which create a new instance, and \__init\__(), which initializes it. 

There are two use cases for a class to define \__new\__():
* The class might be inheriting from a base class whose instances are immutable such as integer, string or tuple those built-in types.
* The class is a metaclass.

In practice, it is **rarely** necessary for a class to define a \__del\__() method. **The only exception** is when the destruction of an object requires a cleanup action such as closing a file, shutting down a network connection, or releasing other system resources. A subtle dange involving object destruction is that instances for which \__del\__() is defined cannot be collected by Python's cyclic garbage collector. Therefore, when a reference cycle is created, a *weak reference* could be used by creating a reference to an object without increasing its reference counts. More information can be found in [weakref module](https://docs.python.org/2/library/weakref.html)


When \__slots\__ of the class is defined, the attribute names that can be assigned on instances are restricted to the names specified. It is a *performance optimization* for both memory and execution speed. If a class inherits from a base class that uses \__slots\__, it also needs to define \__slots\__ for storing its own attributes to take advantage of the benifits \__slots\__ provides.  

## Object Representation

In [8]:
b.__dict__

{'age': 5, 'name': 'Grey'}

From the example, instance *b* is implemented using a dictionary that's accessible as the instance's \__dict\__ attribute.

In [9]:
b.__class__

__main__.OldDog

Instance *b* is linked back to its class by a special attribute \__class\__.

In [10]:
Dog.__dict__.keys()

['__dict__',
 '__module__',
 'inquiry',
 '__del__',
 'num_dogs',
 '__weakref__',
 'bark',
 '__init__',
 '__doc__']

The class *Dog* itself is also a thin layer over a dictionary which can be found in its own \__dict\__ attribute. 

In [11]:
OldDog.__base__

__main__.Dog

Class *OldDog* is linked to its base classes in a special attribute \__base\__.

Then what happen when an attribute is set?

In [12]:
b.age = 6

The special method *b.\__setattr\__("age",6)* is invoked.

Similarly the special method *b.\__delattr\__("age")* is invoked if an attribute is deleted.

In [13]:
print b.age

6


The attribute lookup as above will invoke the special method **b.\__getattrribute\__("age")**, a search process includes checking for properties, looking in the local \__dict\__ attribute, checking the class dictionary, and searching base classes. If search fails, try to invoke the \__getattr\__() method of class. If this fails, an **AttributeError** exception is raised. 

## Metaclasses
A special kind of object called a *metaclass* create and manage the class. 

In [14]:
type(Dog)

type

When a new class is defined with the *class* statement, several things happen:
* The body of the class is executed as a series of statements within its own private dictionary.
* The name is mangling on the private members.
* The name of the class, the list of base classes, and the dictionary are passed to the constructor of a metaclass to create the corresponding class object.

The final step above can be controlled in a number of ways;
* The class can explicitly specify its metaclass by either setting \__metaclass\__ class variable in Python 2, or supplying the *metaclass* keyword argument in the tuple of base class in Python 3.
* If no metaclass is explicitly specified, the class statement examines the first entry in the tuple of base classes.
* If no base classes are specified, the class statement checks for the existence of a global variable called \__metaclass\__. 
* Finally if no \__metaclass\__ value can be found, Python uses the default metaclass. 

The primary use of metaclasses is in frameworks that want to assert more control over the definition of user-defined object. An example can be a metaclass that forces all methods to have a documentation string:

In [15]:
class DocMeta(type):
    def __init__(self, name, bases, dict):
        for key, value in dict.items():
            # Skip special and private methods
            if key.startswith('___'): continue
            # Skip anything not callable
            if not hasattr(value, "__call__"): continue
            # Check for a doc-string
            if not getattr(value, "__doc__"):
                raise TypeError("%s must have a docstring" %key)
        type.__init__(self, name, bases, dict)

The metaclass isn't changing anything about the class that actually gets created but is merely adding some additional checks. 

## Abstract Base Classes
To define an abstract base class, you use [abc](https://docs.python.org/2/library/abc.html) module. This module defines a metaclass (ABCMeta) and a set of decorators (@abstractmethod and @abstractproperty) that are used as follow:

In [16]:
from abc import ABCMeta, abstractmethod, abstractproperty
class sample:
    __metaclass__ = ABCMeta
    @abstractmethod
    def create(self, a, b):
        pass
    @abstractproperty
    def name(self):
        pass

An abstract class enforces rules about methods and properties that must be implemented, and it is useful to programmers who want to **make assertions on the methods and properties** that must be implemented on subclass. Abstract base classes allow preexisting classes to be registered as belonging to that base. When a class is registered with an abstract base, type-checking operations involving the abstract base will return *True* for instances of the registered class.  